In [ ]:
!pip install keras

In [ ]:
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from array import array
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
pd.set_option('display.max_colwidth', 200)

import collections
import helper
import pathlib
import glob
import os


from google.colab import drive

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# function to read raw text file
def read_text(filename):
        # open the file
        file = open(filename, mode='rt', encoding='utf-8')

        # read all text
        text = file.read()
        file.close()
        return text

In [ ]:
#!ls "/content/gdrive/MyDrive/ramayana_1-6.txt"
!ls "/content/gdrive/MyDrive/buddhistSkrt.txt"

In [ ]:
#data = read_text("/content/gdrive/MyDrive/ramayana_1-6.txt")
data = read_text("/content/gdrive/MyDrive/buddhistSkrt.txt")

In [ ]:
data

In [ ]:
# split a text into sentences
def to_lines(text):
      sents = text.strip().split('\n')
      sents = [i.split(',') for i in sents]
      return sents

In [ ]:
eng_san = to_lines(data)
eng_san = np.array(eng_san)
eng_san = eng_san[:]
eng_san

In [ ]:

len(eng_san)

In [ ]:
print(eng_san)

In [ ]:
# Remove Punctuation
eng_san[1] = [s.translate(str.maketrans('', '',  string.punctuation)) for s in eng_san[1]]
eng_san[4] = [s.translate(str.maketrans('', '', string.punctuation)) for s in eng_san[4]]
print(eng_san[0][1])
eng_san[0][4]

In [ ]:
# Converting text to lower case
for i in range(len(eng_san)):
    eng_san[1] = eng_san[1].lower()
    eng_san[4] = eng_san[4].lower()

eng_san[0]

In [ ]:
# Converting Text to Sequence
# empty lists
eng_l = []
san_l = []

# populate the lists with sentence lengths
for i in eng_san[1]:
      eng_l.append(len(i.split()))

for i in eng_san[4]:
      san_l.append(len(i.split()))

length_df = pd.DataFrame({'eng':eng_l, 'san':san_l})

length_df.hist(bins = 30)
plt.show()

In [ ]:
# Build Tokenizer
def tokenization(lines):
      tokenizer = Tokenizer()
      tokenizer.fit_on_texts(lines)
      return tokenizer

In [ ]:
# Prepare English Tokenizer
eng_tokenizer = tokenization(eng_san[1])
eng_vocab_size = len(eng_tokenizer.word_index) + 1

eng_length = 8
print('English Vocabulary Size: %d' % eng_vocab_size)

In [ ]:
# Prepare Spanish Tokenizer
san_tokenizer = tokenization(eng_san[4])
san_vocab_size = len(san_tokenizer.word_index) + 1

san_length = 8
print('Sanskrit Vocabulary Size: %d' % san_vocab_size)

In [ ]:
# Encode and Pad Sequences
def encode_sequences(tokenizer, length, lines):
         # integer encode sequences
         seq = tokenizer.texts_to_sequences(lines)
         # pad sequences with 0 values
         seq = pad_sequences(seq, maxlen=length, padding='post')
         return seq

In [ ]:
# split data into train and test set
train, test = train_test_split(eng_san, test_size=0.2, random_state = 12)

In [ ]:
# prepare training data
trainX = encode_sequences(san_tokenizer, san_length, train[4])
trainY = encode_sequences(eng_tokenizer, eng_length, train[1])

# prepare validation data
testX = encode_sequences(san_tokenizer, san_length, test[4])
testY = encode_sequences(eng_tokenizer, eng_length, test[1])

In [ ]:
# build NMT model
def define_model(in_vocab,out_vocab, in_timesteps,out_timesteps,units):
      model = Sequential()
      model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
      model.add(LSTM(units))
      model.add(RepeatVector(out_timesteps))
      model.add(LSTM(units, return_sequences=True))
      model.add(Dense(out_vocab, activation='softmax'))
      return model

In [ ]:
# model compilation
model = define_model(san_vocab_size, eng_vocab_size, san_length, eng_length, 512)

In [ ]:
rms = optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

In [ ]:
# train model
history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=75, batch_size=512, validation_split = 0.2, verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])
plt.show()

In [ ]:
preds = model.predict_classes(testX.reshape((testX.shape[0],testX.shape[1])))

In [ ]:
def get_word(n, tokenizer):
      for word, index in tokenizer.word_index.items():
          if index == n:
              return word
      return None

In [ ]:
preds_text = []
for i in preds:
       temp = []
       for j in range(len(i)):
            t = get_word(i[j], eng_tokenizer)
            if j > 0:
                if (t == get_word(i[j-1], eng_tokenizer)) or (t == None):
                     temp.append('')
                else:
                     temp.append(t)
            else:
                   if(t == None):
                          temp.append('')
                   else:
                          temp.append(t)

       preds_text.append(' '.join(temp))

In [ ]:
pred_df = pd.DataFrame({'actual' : test[1], 'predicted' : preds_text})

In [ ]:
pred_df.sample()

In [ ]:
def translate(sent):
  user_sents = []
  user_sents.append(sent)
  userTest = encode_sequences(san_tokenizer, san_length, user_sents)
  pred = model.predict_classes(userTest)
  user_text = []
  for i in pred:
        temp = []
        for j in range(len(i)):
              t = get_word(i[j], eng_tokenizer)
              if j > 0:
                  if (t == get_word(i[j-1], eng_tokenizer)) or (t == None):
                      temp.append('')
                  else:
                      temp.append(t)
              else:
                    if(t == None):
                            temp.append('')
                    else:
                            temp.append(t)

        user_text.append(' '.join(temp))
  return user_text[0]